# Training and Prediction with Sagemaker Scikit-learn
This tutorial shows you how to use [Scikit-learn](https://scikit-learn.org/stable/) with Sagemaker by utilizing the pre-built container. Scikit-learn is a popular Python machine learning framework. It includes a number of different algorithms for classification, regression, clustering, dimensionality reduction, and data/feature pre-processing. 

The [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) module  makes it easy to take existing scikit-learn code, which we will show by training a model on the Credit Card dataset and generating a set of predictions. For more information about the Scikit-learn container, see the [sagemaker-scikit-learn-containers](https://github.com/aws/sagemaker-scikit-learn-container) repository and the [sagemaker-python-sdk](https://github.com/aws/sagemaker-python-sdk) repository.

For more on Scikit-learn, please visit the Scikit-learn website: <http://scikit-learn.org/stable/>.

### Table of contents
* [Upload the data for training](#upload_data)
* [Create a Scikit-learn script to train with](#create_sklearn_script)
* [Create the SageMaker Scikit Estimator](#create_sklearn_estimator)
* [Train the SKLearn Estimator on the Credit Card data](#train_sklearn)
* [Using the trained model to make inference requests](#inferece)
 * [Deploy the model](#deploy)
 * [Choose some data and use it for a prediction](#prediction_request)
 * [Endpoint cleanup](#endpoint_cleanup)
* [Batch Transform](#batch_transform)
 * [Prepare Input Data](#prepare_input_data)
 * [Run Transform Job](#run_transform_job)
 * [Check Output Data](#check_output_data)

First, lets create our Sagemaker session and role, and create a S3 prefix to use for the notebook example.

In [2]:
import sagemaker
from sagemaker import get_execution_role

# S3 prefix
bucket_name = 'demo-saeed'
prefix = 'fraudcredit-scikit-est-1'
dataset_name = 'creditcard.csv'

sagemaker_session = sagemaker.Session()

# Get a SageMaker-compatible role used by this Notebook Instance.
role = get_execution_role()

## Upload the data for training <a class="anchor" id="upload_data"></a>

When training large models with huge amounts of data, you'll typically use big data tools, like Amazon Athena, AWS Glue, or Amazon EMR, to create your data in S3. For the purposes of this example, we're using a small size credit card dataset which can load it, write it locally, and then write the dataset to s3 to use.

In [3]:
import numpy as np
import pandas as pd
import os
public_data_url = 's3://saghaboz-datasets/creditcard.csv'
df = pd.read_csv(public_data_url, sep=',',  nrows=1000)
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
import boto3
import s3fs

fs = s3fs.S3FileSystem(anon=False)
def _write_dataframe_to_csv_on_s3_s3fs(dataframe,foldername, filename):
    """ Write a dataframe to a CSV on S3 """

    bytes_to_write = dataframe.to_csv(index=False, header= None).encode()

    # Write buffer to S3 object
    file_path ='s3://{}/{}/{}/{}'.format(bucket_name, prefix,foldername, filename)
    with fs.open(file_path, 'wb') as f:
        f.write(bytes_to_write)
    
    s3_input = sagemaker.s3_input(s3_data=file_path)
    print(s3_input.config)
    return s3_input

_write_dataframe_to_csv_on_s3_s3fs(pd.concat([df.iloc[:,30], df.iloc[:,:-1]],1),'dataset', dataset_name)

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://demo-saeed/fraudcredit-scikit-est-1/dataset/creditcard.csv', 'S3DataDistributionType': 'FullyReplicated'}}}


In [5]:
data_location = 's3://{}/{}/{}/{}'.format(bucket_name, prefix, 'dataset',dataset_name)
df = pd.read_csv(data_location, sep=',',  nrows=1000, header = None)
df.head()


,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,0,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66
3,0,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,...,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50
4,0,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,...,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99


Once we have the data locally, we can use use the tools provided by the SageMaker Python SDK to upload the data to a default bucket. 

In [6]:
s3_input_train = sagemaker.s3_input(s3_data = data_location, content_type='csv')
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-scikit-est-1/dataset/creditcard.csv',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

## Create a Scikit-learn script to train with <a class="anchor" id="create_sklearn_script"></a>
SageMaker can now run a scikit-learn script using the `SKLearn` estimator. When executed on SageMaker a number of helpful environment variables are available to access properties of the training environment, such as:

* `SM_MODEL_DIR`: A string representing the path to the directory to write model artifacts to. Any artifacts saved in this folder are uploaded to S3 for model hosting after the training job completes.
* `SM_OUTPUT_DIR`: A string representing the filesystem path to write output artifacts to. Output artifacts may include checkpoints, graphs, and other files to save, not including model artifacts. These artifacts are compressed and uploaded to S3 to the same S3 prefix as the model artifacts.

Supposing two input channels, 'train' and 'test', were used in the call to the `SKLearn` estimator's `fit()` method, the following environment variables will be set, following the format `SM_CHANNEL_[channel_name]`:

* `SM_CHANNEL_TRAIN`: A string representing the path to the directory containing data in the 'train' channel
* `SM_CHANNEL_TEST`: Same as above, but for the 'test' channel.

A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to model_dir so that it can be hosted later. Hyperparameters are passed to your script as arguments and can be retrieved with an `argparse.ArgumentParser` instance. For example, the script that we will run in this notebook is the below:

```python
import argparse
import pandas as pd
import os

from sklearn import svm
from sklearn.externals import joblib


if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    # Hyperparameters are described here. In this simple example we are just including one hyperparameter.
    parser.add_argument('--max_leaf_nodes', type=int, default=-1)

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output-data-dir', type=str, default=os.environ['SM_OUTPUT_DATA_DIR'])
    parser.add_argument('--model-dir', type=str, default=os.environ['SM_MODEL_DIR'])
    parser.add_argument('--train', type=str, default=os.environ['SM_CHANNEL_TRAIN'])

    args = parser.parse_args()

    # Take the set of files and read them all into a single pandas dataframe
    input_files = [ os.path.join(args.train, file) for file in os.listdir(args.train) ]
    if len(input_files) == 0:
        raise ValueError(('There are no files in {}.\n' +
                          'This usually indicates that the channel ({}) was incorrectly specified,\n' +
                          'the data specification in S3 was incorrectly specified or the role specified\n' +
                          'does not have permission to access the data.').format(args.train, "train"))
    raw_data = [ pd.read_csv(file, header=None, engine="python") for file in input_files ]
    train_data = pd.concat(raw_data)

    # labels are in the first column
    train_y = train_data.ix[:,0]
    train_X = train_data.ix[:,1:]

    # Here we support a single hyperparameter, 'max_leaf_nodes'. Note that you can add as many
    # as your training my require in the ArgumentParser above.
    max_leaf_nodes = args.max_leaf_nodes

    # Now use scikit-learn's decision tree classifier to train the model.
    clf = tree.DecisionTreeClassifier(max_leaf_nodes=max_leaf_nodes)
    clf = clf.fit(train_X, train_y)

    # Print the coefficients of the trained classifier, and save the coefficients
    joblib.dump(clf, os.path.join(args.model_dir, "model.joblib"))


def model_fn(model_dir):
    """Deserialized and return fitted model
    
    Note that this should have the same name as the serialized model in the main method
    """
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf
```

Because the Scikit-learn container imports your training script, you should always put your training code in a main guard `(if __name__=='__main__':)` so that the container does not inadvertently run your training code at the wrong point in execution.

For more information about training environment variables, please visit https://github.com/aws/sagemaker-containers.

## Create SageMaker Scikit Estimator <a class="anchor" id="create_sklearn_estimator"></a>

To run our Scikit-learn training script on SageMaker, we construct a `sagemaker.sklearn.estimator.sklearn` estimator, which accepts several constructor arguments:

* __entry_point__: The path to the Python script SageMaker runs for training and prediction.
* __role__: Role ARN
* __train_instance_type__ *(optional)*: The type of SageMaker instances for training. __Note__: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* __sagemaker_session__ *(optional)*: The session used to train on Sagemaker.
* __hyperparameters__ *(optional)*: A dictionary passed to the train function as hyperparameters.

To see the code for the SKLearn Estimator, see here: https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/sklearn

In [7]:
from sagemaker.sklearn.estimator import SKLearn
models_output_path = 's3://{}/{}/{}'.format(bucket_name, prefix, 'models')
code_location = 's3://{}/{}/{}'.format(bucket_name, prefix, 'source')
script_path = 'scikit_learn_fraud.py'


sklearn = sagemaker.sklearn.estimator.SKLearn(
    entry_point = script_path,
    #train_instance_type = "ml.c4.xlarge",
    train_instance_type = "local",
    role=role,
    code_location = code_location,
    output_path = models_output_path,
    #sagemaker_session = sagemaker_session,
    hyperparameters = {'max_leaf_nodes': 30})

## Train SKLearn Estimator on yout data <a class="anchor" id="train_sklearn"></a>
Training is very simple, just call `fit` on the Estimator! This will start a SageMaker Training job that will download the data for us, invoke our scikit-learn code (in the provided script file), and save any model artifacts that the script creates.

In [8]:
s3_input_train.config

{'DataSource': {'S3DataSource': {'S3DataType': 'S3Prefix',
   'S3Uri': 's3://demo-saeed/fraudcredit-scikit-est-1/dataset/creditcard.csv',
   'S3DataDistributionType': 'FullyReplicated'}},
 'ContentType': 'csv'}

In [9]:
sklearn.fit({'train': s3_input_train})

Creating tmpzf733x9q_algo-1-asmz6_1 ... 
Attaching to tmpzf733x9q_algo-1-asmz6_12mdone
algo-1-asmz6_1  | 2020-02-28 17:12:21,913 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
algo-1-asmz6_1  | 2020-02-28 17:12:21,915 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-asmz6_1  | 2020-02-28 17:12:21,927 sagemaker_sklearn_container.training INFO     Invoking user training script.
algo-1-asmz6_1  | 2020-02-28 17:12:22,048 sagemaker-containers INFO     Module scikit_learn_fraud does not provide a setup.py. 
algo-1-asmz6_1  | Generating setup.py
algo-1-asmz6_1  | 2020-02-28 17:12:22,049 sagemaker-containers INFO     Generating setup.cfg
algo-1-asmz6_1  | 2020-02-28 17:12:22,049 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-asmz6_1  | 2020-02-28 17:12:22,049 sagemaker-containers INFO     Installing module with the following command:
algo-1-asmz6_1  | /miniconda3/bin/python -m pip install . 
algo-1-asmz6_1  

In [10]:
sklearn.latest_training_job.job_name

'sagemaker-scikit-learn-2020-02-28-17-12-19-798'

## Using the trained model to make inference requests <a class="anchor" id="inference"></a>

### Deploy the model <a class="anchor" id="deploy"></a>

Deploying the model to SageMaker hosting just requires a `deploy` call on the fitted model. This call takes an instance count and instance type.

In [13]:
predictor = sklearn.deploy(initial_instance_count=1, instance_type="ml.m4.xlarge")
#predictor = sklearn.deploy(initial_instance_count=1, instance_type="local")

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 611, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 672, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 616, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmp564u65co/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1



RuntimeError: Giving up, endpoint didn't launch correctly

In [12]:
predictor.endpoint

'sagemaker-scikit-learn-2019-07-02-13-50-52-792'

### Choose some data and use it for a prediction <a class="anchor" id="prediction_request"></a>

In order to do some predictions, we'll extract some of the data we used for training and do predictions against it. This is, of course, bad statistical practice, but a good way to see how the mechanism works.

In [13]:
test_X = df.iloc[:,:-1]
print(test_X.shape)
print(predictor.predict(test_X.values))

(1000, 30)
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0

Prediction is as easy as calling predict with the predictor we got back from deploy and the data we want to do predictions with. The output from the endpoint return an numerical representation of the classification prediction; which are the labels. We can compare against the original label that we parsed.

### Endpoint cleanup <a class="anchor" id="endpoint_cleanup"></a>

When you're done with the endpoint, you'll want to clean it up.

In [14]:
# sklearn.delete_endpoint()

## Batch Transform <a class="anchor" id="batch_transform"></a>
We can also use the trained model for asynchronous batch inference on S3 data using SageMaker Batch Transform.

In [31]:
# Define a SKLearn Transformer from the trained SKLearn Estimator
transform_output_path = 's3://{}/{}/{}'.format(bucket_name, prefix, 'transform-output')
transformer = sklearn.transformer(instance_count=1, instance_type='ml.m4.xlarge', output_path = transform_output_path)

### Prepare Input Data <a class="anchor" id="prepare_input_data"></a>
We will extract 10 random samples of 100 rows from the training data, then split the features (X) from the labels (Y). Then upload the input data to a given location in S3.

In [32]:
%%bash
mkdir -p data

In [33]:
df.iloc[:,1] = pd.to_numeric(df.iloc[:,1], downcast='float')
export = df.to_csv('data/cc_t.csv',index=False)

In [34]:
%%bash
# Randomly sample the dataset 10 times, then split X and Y
mkdir -p batch_data/XY batch_data/X batch_data/Y
for i in {0..9}; do
    cat data/cc_t.csv | shuf -n 100 > batch_data/XY/cc_sample_${i}.csv
    cat batch_data/XY/cc_sample_${i}.csv | cut -d',' -f2- > batch_data/X/cc_sample_X_${i}.csv
    cat batch_data/XY/cc_sample_${i}.csv | cut -d',' -f1 > batch_data/Y/cc_sample_Y_${i}.csv
done

In [35]:
# Upload input data from local filesystem to S3
batch_input_s3 = sagemaker_session.upload_data('batch_data/X', bucket =bucket_name,  key_prefix=prefix + '/batch_input')

### Run Transform Job <a class="anchor" id="run_transform_job"></a>
Using the Transformer, run a transform job on the S3 input data.

In [36]:
batch_input_s3

's3://demo-saeed/fraudcredit-scikit-est-1/batch_input'

In [37]:
# Start a transform job and wait for it to finish
transform_output_path = 's3://{}/{}/{}'.format(bucket_name, prefix, 'transform_output')
transformer.transform(batch_input_s3, content_type='text/csv' )
print('Waiting for transform job: ' + transformer.latest_transform_job.job_name)
transformer.wait()

Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 611, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 672, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 1

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 616, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpcraq66l_/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1



KeyboardInterrupt: 

### Check Output Data  <a class="anchor" id="check_output_data"></a>
After the transform job has completed, download the output data from S3. For each file "f" in the input data, we have a corresponding file "f.out" containing the predicted labels from each input row. We can compare the predicted labels to the true labels saved earlier.

In [23]:
# Download the output data from S3 to local filesystem
batch_output = transformer.output_path
!mkdir -p batch_data/output
!aws s3 cp --recursive $batch_output/ batch_data/output/
# Head to see what the batch output looks like
!head batch_data/output/*

download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_1.csv.out to batch_data/output/cc_sample_X_1.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_0.csv.out to batch_data/output/cc_sample_X_0.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_2.csv.out to batch_data/output/cc_sample_X_2.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_3.csv.out to batch_data/output/cc_sample_X_3.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_4.csv.out to batch_data/output/cc_sample_X_4.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_7.csv.out to batch_data/output/cc_sample_X_7.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample_X_6.csv.out to batch_data/output/cc_sample_X_6.csv.out
download: s3://demo-saeed/fraudcredit-scikit-est2/transform-output/cc_sample

In [24]:
%%bash
# For each sample file, compare the predicted labels from batch output to the true labels
for i in {1..9}; do
    diff -s batch_data/Y/cc_sample_Y_${i}.csv \
        <(cat batch_data/output/cc_sample_X_${i}.csv.out | sed 's/[["]//g' | sed 's/, \|]/\n/g') \
        | sed "s/\/dev\/fd\/63/batch_data\/output\/cc_sample_X_${i}.csv.out/"
done

Files batch_data/Y/cc_sample_Y_1.csv and batch_data/output/cc_sample_X_1.csv.out are identical
Files batch_data/Y/cc_sample_Y_2.csv and batch_data/output/cc_sample_X_2.csv.out are identical
64c64
< 0
---
> 1
Files batch_data/Y/cc_sample_Y_4.csv and batch_data/output/cc_sample_X_4.csv.out are identical
Files batch_data/Y/cc_sample_Y_5.csv and batch_data/output/cc_sample_X_5.csv.out are identical
Files batch_data/Y/cc_sample_Y_6.csv and batch_data/output/cc_sample_X_6.csv.out are identical
Files batch_data/Y/cc_sample_Y_7.csv and batch_data/output/cc_sample_X_7.csv.out are identical
Files batch_data/Y/cc_sample_Y_8.csv and batch_data/output/cc_sample_X_8.csv.out are identical
Files batch_data/Y/cc_sample_Y_9.csv and batch_data/output/cc_sample_X_9.csv.out are identical
